In [4]:
import scanpy as sc
from SpaceFlow import SpaceFlow
from utils import mk_dir
import os
import pandas as pd

In [5]:
def load_ST_file(file_fold, count_file='filtered_feature_bc_matrix.h5', load_images=True, file_Adj=None):
    adata_h5 = sc.read_visium(file_fold, load_images=load_images, count_file=count_file)
    adata_h5.var_names_make_unique()

    if load_images is False:
        if file_Adj is None:
            file_Adj = os.path.join(file_fold, "spatial/tissue_positions_list.csv")

        positions = pd.read_csv(file_Adj, header=None)
        positions.columns = [
            'barcode',
            'in_tissue',
            'array_row',
            'array_col',
            'pxl_col_in_fullres',
            'pxl_row_in_fullres',
        ]
        positions.index = positions['barcode']
        adata_h5.obs = adata_h5.obs.join(positions, how="left")
        adata_h5.obsm['spatial'] = adata_h5.obs[['pxl_row_in_fullres', 'pxl_col_in_fullres']].to_numpy()
        adata_h5.obs.drop(columns=['barcode', 'pxl_row_in_fullres', 'pxl_col_in_fullres'], inplace=True)

    print('adata: (' + str(adata_h5.shape[0]) + ', ' + str(adata_h5.shape[1]) + ')')
    return adata_h5

In [6]:
import sys
sys.path.append('../')
import utils_for_all as usa
if __name__ == '__main__':

    results = pd.DataFrame()
    dataset1 = ["Stereo","Breast_cancer", "Mouse_brain", "STARmap","SeqFish","STARmap"]
    Dataset_test = ['151673']

for dataset in  Dataset_test:
    print(f"====================begin test on {dataset}======================================")

    if dataset.startswith('15'):
        save_path = f'../../Output/SpaceFlow/DLPFC/{dataset}/'
    else:
        save_path = f'../../Output/SpaceFlow/{dataset}/'
    mk_dir(save_path)

    results = pd.DataFrame()
    import psutil, time, tracemalloc
    for i in range(1):
        num = i + 1
        print("===Training epoch:{}====".format(num))
        start = time.time()
        tracemalloc.start()
        start_MB = psutil.Process(os.getpid()).memory_info().rss / 1024 / 1024 / 1024
        adata_h5,n_cluster = usa.get_adata(dataset, data_path='../../Dataset/')

        sf = SpaceFlow.SpaceFlow(count_matrix=adata_h5.X,
                                 spatial_locs=adata_h5.obsm['spatial'])

        sf.preprocessing_data()
        sf.train(embedding_save_filepath=save_path,epochs=1000)

        print("train finished，begin segment！")
        res=sf.segmentation_1(adata_h5, dataset, save_path)

        end = time.time()
        end_MB = psutil.Process(os.getpid()).memory_info().rss / 1024 / 1024 / 1024
        uesd_time = end - start
        used_memo = end_MB - start_MB
        current, peak = tracemalloc.get_traced_memory()
        tracemalloc.stop()
        peak = peak / 1024.0 / 1024.0 / 1024.0
        print(u'Current memory usage_end:：%.4f GB' % used_memo)
        print('time: {:.4f} s'.format(uesd_time))
        print('memory blocks peak:{:>10.4f} GB'.format(peak))
        tracemalloc.clear_traces()
        res["time"] = uesd_time
        res["Memo"] = used_memo
        res["Memo_peak"] = peak
        res["round"] = i + 1
        results = results._append(res, ignore_index=True)

print(results.head())
results.to_csv(f'{save_path}result.csv')
results.set_index('dataset', inplace=True)

res_mean = results.mean()
res_mean.to_csv(f'{save_path}{dataset}_mean.csv', header=True)
res_std = results.std()
res_std.to_csv(f'{save_path}{dataset}_std.csv', header=True)
res_median = results.median()
res_median.to_csv(f'{save_path}{dataset}_median.csv', header=True)


====================begin test on 151673======================================
===Training epoch:1====
load DLPFC dataset:


D:\Anaconda_install\envs\STAGATE\lib\site-packages\anndata\_core\anndata.py:1830: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
D:\Anaconda_install\envs\STAGATE\lib\site-packages\SpaceFlow\SpaceFlow.py:89: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  self.adata = anndata.AnnData(count_matrix.astype(float)) ##将csc矩阵变成adata


Epoch 2/1000, Loss: 1.5077831745147705
Epoch 12/1000, Loss: 1.3804250955581665
Epoch 22/1000, Loss: 1.0224157571792603
Epoch 32/1000, Loss: 0.6598470211029053
Epoch 42/1000, Loss: 0.4666133522987366
Epoch 52/1000, Loss: 0.36088719964027405
Epoch 62/1000, Loss: 0.27049607038497925
Epoch 72/1000, Loss: 0.2387990951538086
Epoch 82/1000, Loss: 0.22434791922569275
Epoch 92/1000, Loss: 0.1979411095380783
Epoch 102/1000, Loss: 0.18085426092147827
Epoch 112/1000, Loss: 0.1655007302761078
Epoch 122/1000, Loss: 0.14709308743476868
Epoch 132/1000, Loss: 0.15572628378868103
Epoch 142/1000, Loss: 0.1282215416431427
Epoch 152/1000, Loss: 0.13188980519771576
Epoch 162/1000, Loss: 0.1326189488172531
Epoch 172/1000, Loss: 0.11756554245948792
Epoch 182/1000, Loss: 0.10986325889825821
Epoch 192/1000, Loss: 0.11024940758943558
Epoch 202/1000, Loss: 0.12453519552946091
Epoch 212/1000, Loss: 0.10755883157253265
Epoch 222/1000, Loss: 0.10025531053543091
Epoch 232/1000, Loss: 0.1007167249917984
Epoch 242/1000